In [ ]:
print("Insert history penumpang ke hist_pax_revenue")

In [1]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from datetime import date, time, datetime
import math

In [3]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [4]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [5]:
# Accommodate query
sql = """SELECT id_port, code
FROM stg_pelni.master_port
where flag_ppss like 'Y'"""

In [6]:
# Read Query
c1 = sqlio.read_sql_query(sql, conn)
d1 = sqlio.read_sql_query(sql, conn)

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_7408\1901621261.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  c1 = sqlio.read_sql_query(sql, conn)
C:\Users\BN001602001\AppData\Local\Temp\ipykernel_7408\1901621261.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  d1 = sqlio.read_sql_query(sql, conn)


In [7]:
# Read CSV
df1 = pd.read_excel('data_hist_pax/2010.xlsx')
df2 = pd.read_excel('data_hist_pax/2011.xlsx')
df3 = pd.read_excel('data_hist_pax/2012.xlsx')
df4 = pd.read_excel('data_hist_pax/2013.xlsx')
df5 = pd.read_excel('data_hist_pax/2014.xlsx')
df6 = pd.read_excel('data_hist_pax/2015.xlsx')
df7 = pd.read_excel('data_hist_pax/2016.xlsx')

In [8]:
df8 = pd.read_excel('data_hist_pax/2017.xlsx')
df9 = pd.read_excel('data_hist_pax/2023.xlsx')
df = pd.concat([df8, df9])

In [9]:
df = df.rename(columns={'SHIP_CODE':'kode_kapal', 'SHIP_NAME': 'nama_kapal', 'DEP_DATE':'tanggal', 'ORG': 'org',
                          'DES' : 'des','PAX': 'pax', 'AMOUNT' :'revenue','ORG_CODE':'kode_org','DES_CODE':'kode_des',
                          'TRANSIT':'transit'})

df = df.loc[:, ['org', 'kode_org', 'des', 'kode_des', 'revenue','tanggal', 'pax', 'transit', 'kode_kapal', 'nama_kapal']]

In [10]:
# Join data used concat
cdf = pd.concat([df1, df2])
cdf = pd.concat([cdf, df3])
cdf = pd.concat([cdf, df4])
cdf = pd.concat([cdf, df5])
cdf = pd.concat([cdf, df6])
cdf = pd.concat([cdf, df7])



In [11]:
cdf['date'] = cdf['tanggal'].str.split(' ').str[0]
cdf['time'] = cdf['tanggal'].str.split(' ').str[1]

In [12]:
cdf['date']= list(map(lambda x: dt.datetime.strptime(x,'%d-%m-%Y').strftime('%Y-%m-%d'), cdf['date']))

cdf['a'] = cdf['time'].str.split('.').str[0]
cdf['b'] = cdf['time'].str.split('.').str[1]


cdf['time'] = cdf['a'] + ':' + cdf['b'] + ':00'

cdf['date'] = pd.to_datetime(cdf['date']).dt.date
cdf['time'] = pd.to_datetime(cdf['time']).dt.time

cdf['tanggal'] = [dt.datetime.combine(a, b) for a, b in zip(cdf['date'], cdf['time'])]


C:\Users\BN001602001\AppData\Local\Temp\ipykernel_7408\446524263.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cdf['time'] = pd.to_datetime(cdf['time']).dt.time


In [13]:
cdf = pd.concat([cdf, df])

In [14]:
cdf = cdf.loc[:, ['org', 'kode_org', 'des', 'kode_des', 'revenue','tanggal', 'pax', 'transit', 'kode_kapal', 'nama_kapal']]

In [15]:
cdf['tanggal'] = pd.to_datetime(cdf['tanggal'])

In [16]:
# ALL
# Change columns name
c1 = c1.rename(columns={'code': 'des_code'})
cdf = cdf.rename(columns={'kode_des': 'des_code'})

# Mapping for code_des 
cdf['des_code'] = cdf['des_code'].astype(int)
c1['des_code'] = c1['des_code'].astype(int)

# Inner join
inner_join1 = pd.merge(cdf, 
                      c1, 
                      on ='des_code', 
                      how ='inner')

# Change columns name
inner_join1 = inner_join1.rename(columns={'id_port': 'destination_port'})
inner_join1 = inner_join1.rename(columns={'kode_org': 'org_code'})
d1 = d1.rename(columns={'code': 'org_code'})

d1['org_code'] = d1['org_code'].astype(int)
inner_join1['org_code'] = inner_join1['org_code'].astype(int)

# Inner join
merge_data1 = pd.merge(inner_join1, 
                      d1, 
                      on ='org_code',
                      how ='inner')
cleandf = merge_data1.rename(columns={'id_port': 'origin_port'})
cleandf

,org,org_code,des,des_code,revenue,tanggal,pax,transit,kode_kapal,nama_kapal,destination_port,origin_port
0,TANJUNG PRIOK,431,BELAWAN,144,46813000,2010-10-15 10:00:00,100,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b
1,TANJUNG PRIOK,431,BELAWAN,144,30376500,2010-10-22 10:00:00,94,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b
2,TANJUNG PRIOK,431,BELAWAN,144,35871500,2010-10-29 10:00:00,87,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b
3,TANJUNG PRIOK,431,BELAWAN,144,52396500,2010-11-26 10:00:00,139,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b
4,TANJUNG PRIOK,431,BELAWAN,144,39814000,2010-12-03 10:00:00,122,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b
...,...,...,...,...,...,...,...,...,...,...,...,...
376216,MOA,959,TEPA,958,16538000,2022-09-18 00:00:00,246,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156
376217,MOA,959,TEPA,958,15190000,2022-10-02 00:00:00,217,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156
376218,MOA,959,TEPA,958,15068000,2022-12-03 00:00:00,225,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156
376219,MOA,959,TEPA,958,37538000,2022-12-17 00:00:00,546,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156


In [ ]:
# ============================================ DATETIME =====================================

In [ ]:
cleandf['tanggal'] = cleandf['tanggal'].astype(str)

In [ ]:
# Separate beased on index (between date with time)
cleandf['jam'] = cleandf['tanggal'].str.split(' ').str[1]
cleandf['bulan'] = cleandf['tanggal'].str.split(' ').str[0]


# Separate beased on index for date
cleandf['d_1'] = cleandf['bulan'].str.split('-').str[0]
cleandf['b_1'] = cleandf['bulan'].str.split('-').str[1]
cleandf['y_1'] = cleandf['bulan'].str.split('-').str[2]

In [ ]:
# Convert to Interger (in calculation + typedata must integer)
cleandf['y_1'] = cleandf['y_1'].astype(int)

# Calculation date + 7 years
temp = []
for index, row in cleandf.iterrows():
    rs = row['y_1'] + 7
    temp.append(rs)
    
cleandf['y'] = temp

# Convert to string
cleandf['y'] = cleandf['y'].astype(str)

# Combine index date
cleandf['temp_date'] = cleandf['y'] +'-'+ cleandf['b_1'] +'-'+ cleandf['d_1']

In [ ]:
# Combine data
cleandf['y'] = cleandf['y'].astype(str)
cleandf['temp_date'] = cleandf['y'] +'-'+ cleandf['b_1'] +'-'+ cleandf['d_1']

In [ ]:
# Checked Data
cleandf.loc[cleandf.index == 308]

In [ ]:
# separate beased on index
cleandf['j_1'] = cleandf['jam'].str.split('.').str[1]
cleandf['j_2'] = cleandf['jam'].str.split('.').str[0]

# Combine index time
cleandf['temp_jam'] = cleandf['j_2'] +':'+ cleandf['j_1']

# Convert to date & time
cleandf['date'] = pd.to_datetime(cleandf['temp_date'], format='%Y-%m-%d', errors='coerce')
# cleandf['date'] = cleandf['temp_date'].astype(str)
cleandf['time'] = pd.to_datetime(cleandf['temp_jam']).dt.time

In [ ]:
cleandf

In [ ]:
# Combine between date with time to datetime
cleandf['departure_date'] = [dt.datetime.combine(a, b) for a, b in zip(cleandf['date'], cleandf['time'])]

In [ ]:
# Loc columns
cleandf = cleandf.loc[:, ['kode_kapal', 'nama_kapal', 'tanggal','org_code', 'org', 'des_code', 'des', 'pax', 'revenue', 'transit', 'destination_port', 'origin_port']]
cleandf.head()

In [ ]:
# Change name columns
cleandf = cleandf.rename(columns={'kode_kapal':'ship_code', 'nama kapal':'ship_name', 'org':'org_name','des':'des_name','pax':'total', 'transit': 'stop', 'tanggal': 'departure_date'})

In [17]:
# Add columns
cleandf['type_rev'] = '0'
cleandf['type'] = 'PASSENGER'

In [18]:
cleandf

,org,org_code,des,des_code,revenue,tanggal,pax,transit,kode_kapal,nama_kapal,destination_port,origin_port,type_rev,type
0,TANJUNG PRIOK,431,BELAWAN,144,46813000,2010-10-15 10:00:00,100,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b,0,PASSENGER
1,TANJUNG PRIOK,431,BELAWAN,144,30376500,2010-10-22 10:00:00,94,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b,0,PASSENGER
2,TANJUNG PRIOK,431,BELAWAN,144,35871500,2010-10-29 10:00:00,87,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b,0,PASSENGER
3,TANJUNG PRIOK,431,BELAWAN,144,52396500,2010-11-26 10:00:00,139,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b,0,PASSENGER
4,TANJUNG PRIOK,431,BELAWAN,144,39814000,2010-12-03 10:00:00,122,2.0,119,KELUD,37463b21-4d37-4327-839a-56985cdc8aa0,b2674a83-0456-4666-893d-7c7522f9232b,0,PASSENGER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376216,MOA,959,TEPA,958,16538000,2022-09-18 00:00:00,246,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156,0,PASSENGER
376217,MOA,959,TEPA,958,15190000,2022-10-02 00:00:00,217,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156,0,PASSENGER
376218,MOA,959,TEPA,958,15068000,2022-12-03 00:00:00,225,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156,0,PASSENGER
376219,MOA,959,TEPA,958,37538000,2022-12-17 00:00:00,546,0.0,120,KM.PANGRANGO,ea569719-3919-41cc-a2ac-c2e3cb234930,5e427eb6-c2bc-4257-bf39-aee0eadc1156,0,PASSENGER


In [21]:
cleandf.to_csv('data_hist_pax/hist_pax_total.csv')